In [4]:
%config IPCompleter.greedy=True

In [2]:
import requests 
import json
import pandas as pd
import warnings
import pyecharts as pe

ERROR:lml.utils:failed to import pyecharts_snapshot
Traceback (most recent call last):
  File "/Users/chaozy/anaconda3/lib/python3.6/site-packages/lml/utils.py", line 43, in do_import
    plugin_module = __import__(plugin_module_name)
ModuleNotFoundError: No module named 'pyecharts_snapshot'


In [3]:
# warnings will not pop up
warnings.filterwarnings("ignore")
# GET Access Token
host = 'https://aip.baidubce.com/oauth/2.0/token?grant_type=client_credentials&client_id=E52brsYEkEv9bG7xwqecY64S&client_secret=BAaNBmr3gV7PHIfMFbYBd3QVfT4sFQC9'
response = requests.get(host)
if response:
    access_token = response.json()['access_token']
access_token

'24.da7050c22a3b2d5ed7d94bad9a1b198f.2592000.1589352409.282335-19192750'

In [4]:
# Fill in header parameters
headers = {'content-type': 'application/json'}

# 生成url
url = 'https://aip.baidubce.com/rpc/2.0/nlp/v1/lexer?charset=UTF-8&access_token=' +  access_token

In [5]:
# read file
with open("SunYung.txt", 'r', encoding="utf-8") as file:
    data = file.read().replace('\n', '')

In [6]:
# perform word analysis on this article
response = requests.post(url, 
                         data = json.dumps({'text': data}),    # news变量写在这里
                         headers = headers)
if response:
    result = response.json()

df_news_pos = pd.DataFrame(result['items'])[['item', 'pos', 'ne']]
df_news_pos.head()  # defaultly show first five data if no value is given

,item,pos,ne
0,孙杨,,PER
1,的,u,
2,事情,n,
3,，,w,
4,还,d,


In [7]:
df_nper = df_news_pos[(df_news_pos["pos"] == 'n') | (df_news_pos["ne"] == "PER")]
df_nper.head()

,item,pos,ne
0,孙杨,,PER
2,事情,n,
12,妈妈,n,
13,杨明,,PER
19,舆论,n,


In [8]:
# word Counts
nper_counts = df_nper["item"].value_counts()
nper_counts.head()

孙杨    70
杨明    26
妈妈    15
事情    11
儿子     9
Name: item, dtype: int64

In [9]:
# Generate Word Cloud object
wordCloud = pe.WordCloud("Sun Yung")

In [10]:
# GET every words
words = nper_counts.index
# GET counts of every word
word_counts = nper_counts.values

In [11]:
# DRAW the word cloud
wordCloud.add("Word Count", words, word_counts, shape = "diamond", word_size_range=[20, 100], rotate_step=10 )
wordCloud